In [2]:
from environment import Environment
from individual import BinaryIndividual 
from crossover import SinglePointCrossOver
from selector import LeadingSelector
from fitness import WeightedSumFitness
import logging 
import numpy as np

# Formulate the Problem
Define the genome as a 0-1 vector to represent each box. The problem is to find out a vector that maximize sum of box ***I***mportance where sum of box ***W***eights are less than or equal to 250. eq.   
$$ \max \sum_{i}{I_i} \quad where \quad i \in \{0, \dots, 11\}  $$
$$ s.t. \sum_{i}{W_i} <= 250 $$

# Define the components 
1. Chromosome: binary vector to represent whether the box is selected to put into my bag. eg. [1,1,0,0,0,0,0,0,0,1,1,0]
2. Importance: a vector with same length and constant values: [6, 5, 8, 7, 6, 9, 4, 5, 4, 9, 2, 1]
3. Box Weight: a vector with same length and constant values: [20, 30, 60, 90, 50, 70, 30, 30, 70, 20, 20, 60]
2. Fitness: sum of importance, because we want to add as much important boxes as possible
3. Constraints: sum of box weights(this is the weight of the boxes, but not weight parameter) should be less than or equal to 250.
4. Survive Ratio: 0.5(or cull 0.5 as predefined by assignment requirement)
5. Mutation Rate and Crossover operator can be adjusted in the experiment. 


In [3]:
def total_size(individual, size=np.array([])) :
    chr_arr = np.array(individual.chromosome)
    siz_arr = np.array(size)
    total = np.dot(chr_arr, siz_arr.T)
    return total 

def total_size_lt250(individual, size=np.array([])) :
    total = total_size(individual, size)
    return total <= 250

# Define Hyperparameters 
- Initial Individuals: randomly initialized individuals will involve more possibility to find the best solution.
- Selection ratio: can be changed to a smaller value to make the algorithm faster and a larger value to keep more candidates. 
- Crossover: crossover strategy can be customized, here only Single Point Crossover is implemented

# Algorithm Steps
1. Initialize: input parameters and create algorithm instance
1. Calculate fitness: calculate fitness value for every individual
1. Select: keep only the individuals fulfill the constrants
1. Reproduce: generate new individuals by mutation and crossover operator
1. Check Stop: check if stop criteria is fulfilled. If yes, stop the progress, otherwise repeat from step 2.  

In [4]:
box_importance = [6, 5, 8, 7, 6, 9, 4, 5, 4, 9, 2, 1]
box_weights = [20, 30, 60, 90, 50, 70, 30, 30, 70, 20, 20, 60]


In [5]:
individuals = [ 
    BinaryIndividual([1,1,1,0,0,0,0,0,0,0,0,1],0,0),
    BinaryIndividual([1,0,0,0,1,0,0,0,0,0,0,1],0,0),
    BinaryIndividual([0,0,0,0,0,1,1,0,0,1,0,0],0,0),
    BinaryIndividual([0,0,1,0,0,0,0,0,1,0,0,1],0,0),
    BinaryIndividual([0,1,0,0,1,0,0,0,0,0,0,1],0,0),
]    

In [6]:
sel = LeadingSelector(
    ratio = 0.5,
    constraints=[lambda x: total_size_lt250(x, box_weights)]
)

fit = WeightedSumFitness(weights = box_importance)
xo = SinglePointCrossOver()

In [7]:
individuals[0]

In [8]:
env = Environment(
    individuals,
    selector=sel,
    crossover=xo, 
    fitness_func=fit,
    MAX_GENERATION=50,
    CAPACITY=100, 
    MAX_ITERATION=100,
    log_level=logging.DEBUG
)

env.evolute()

print(env.species.population(), env.species.generations())


2022-11-09 11:56:47,769 - Species - INFO - ITERATION START -- : 0
2022-11-09 11:56:54,680 - Species - INFO - ITERATION START -- : 5
2022-11-09 11:57:09,279 - Species - INFO - ITERATION START -- : 10
2022-11-09 11:57:24,818 - Species - INFO - ITERATION START -- : 15
2022-11-09 11:57:40,534 - Species - INFO - ITERATION START -- : 20
2022-11-09 11:57:57,000 - Species - INFO - ITERATION START -- : 25
2022-11-09 11:58:11,383 - Species - INFO - ITERATION START -- : 30
2022-11-09 11:58:25,625 - Species - INFO - ITERATION START -- : 35
2022-11-09 11:58:43,547 - Species - INFO - ITERATION START -- : 40
2022-11-09 11:58:56,358 - Species - INFO - ITERATION START -- : 45
2022-11-09 11:59:13,197 - Species - INFO - ITERATION START -- : 50
2022-11-09 11:59:22,290 - Species - INFO - MAX_GENERATION Achieved -- top fitness: 44


82 51


# Recap
After 50 iterations run, the maximum fitness reaches 44 and survived population is 53 at last round.  
Checking with the original problem settings, our selected combination is {1,2,3,6,8,10,11}, with vector representation as [1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0].  
This set reaches 250 capability with highest total importance 44. It shows we find the combination as much valuable as possible without exceeding the backpack capability. 
## packages:
python>=3.7  
numpy==1.19.5  
pandas==1.1.1 

In [9]:
print('The best 3 solutions are: ')
for sol in env.getSolution(3) :
    print(sol) 


The best 3 solutions are: 
[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0]
